# 2001 Hurricane Season Impact on MTA Ridership 
**CLIENT & NEED**

*Client:* NYC Extreme Weather Task Force

*Need:* Climate change is increasing extreme weather patterns in NYC, and the Extreme Weather Task Force was recently launched following the deadly impacts of Tropical Storm Ida in 2021 to improve citywide planning and response to these events. The Task Force needs transit data on storm system events to target MTA capital improvements and emergency planning strategies.

**GOAL**

For 2021 hurricane season ID stations with highest impacts to service during and following storm events

**DATA**

- MTA Turnstile Data for August - October 2021 (http://web.mta.info/developers/turnstile.html)
- National Weather Service Storm Warning Data for New York for August - October 2021 (https://nwschat.weather.gov/lsr/#OKX/202108010700/202111010659/0100)

## A. Data Import & Cleaning

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sqlalchemy import create_engine

### A.1 Import & Clean Data: MTA
**Pull turnstile data from MTA for August - October 2021**

*Citation: referred to MTA exercise solution code to support data pull and cleanup*

In [4]:
engine = create_engine('sqlite:///my-sqlite.db')
engine.table_names

In [ ]:
def mtapull(week_lst):
    url = "http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt"
    mta_dfs = []
    for week in week_lst:
        mta_url = url.format(week)
        mta_dfs.append(pd.read_csv(mta_url, parse_dates = [['DATE','TIME']], keep_date_col = True))
    return pd.concat(mta_dfs)

weeks = ['211030','211023','211016','211009','211002','210925','210918','210911','210904','210828','210821','210814','210807','210731']
mta_df = mtapull(weeks)

In [ ]:
#clean columns
mta_df.columns = mta_df.columns.str.lower()
mta_df.columns = mta_df.columns.str.strip()
mta_df.rename(columns={'c/a':'c-a'}, inplace = True)
mta_df.columns

In [ ]:
#create unique id for each stile
mta_df["stile_id"] = mta_df["station"] + mta_df['scp'] + mta_df['unit'] + mta_df['c-a']

In [ ]:
#check for correct number of weeks/months
mta_df.date.value_counts().sort_index()

In [ ]:
#check for dupes
mta_df["desc"].value_counts()

In [ ]:
mta_df.duplicated(subset = ['date_time','stile_id']).value_counts()

In [ ]:
mta_df.groupby(["stile_id", "date_time"]).entries.count().reset_index().sort_values("entries", ascending=False).head(5)

In [ ]:
mta_df[mta_df.duplicated(subset = ['date_time','stile_id']) == True]['desc'].value_counts()

In [ ]:
mta_df[
    (mta_df['stile_id'] == 'ST. GEORGE00-00-02R070S101') &
    (mta_df['date'] == '09/15/2021')
]

In [ ]:
#drop dupes
mta_df.drop_duplicates(subset=["stile_id", "date_time"],inplace=True)

In [ ]:
mta_df[
    (mta_df['stile_id'] == 'ST. GEORGE00-00-02R070S101') &
    (mta_df['date'] == '09/14/2021')
]

In [ ]:
mta_df[mta_df.duplicated(subset = ['date_time','stile_id']) == True]['desc'].value_counts()

In [ ]:
mta_df.groupby(["stile_id", "date_time"]).entries.count().reset_index().sort_values("entries", ascending=False).head(5)

In [ ]:
#create daily stile data table
stiles_daily_df = (mta_df.groupby(['stile_id','c-a','unit','scp','station','date'],as_index=False)[['entries','exits']].last())

In [ ]:
stiles_daily_df

In [ ]:
#create daily stile entry/exit columns
#shift columns
stiles_daily_df[['prev_date','prev_entries','prev_exits']] = (stiles_daily_df
                                                              .groupby('stile_id')[['date','entries','exits']]
                                                              .apply(lambda grp: grp.shift(1)))

In [ ]:
#check that shift is correct
stiles_daily_df[stiles_daily_df['date'] == '07/24/2021'].info()

In [ ]:
#drop NaN rows
stiles_daily_df.dropna(subset = ['prev_date'], inplace = True)

In [ ]:
#create daily entry / exit columns
stiles_daily_df.head()

In [ ]:
#check for reversed values
stiles_daily_df[
    (stiles_daily_df['prev_entries']) > (stiles_daily_df['entries'])
].tail()

In [ ]:
stiles_daily_df[
    (stiles_daily_df['prev_entries']) > (stiles_daily_df['entries'])
].groupby('stile_id').size()

In [ ]:
#function to replace reversed values and reset counter values
def get_daily_counts(row, max_counter, stile_col, prev_stile_col):
    counter = row[stile_col] - row[prev_stile_col]
    if counter < 0:
        # reversed counter
        counter = -counter
    if counter > max_counter:
        # reset counter
        print(row[stile_col], row[prev_stile_col])
        counter = min(row[stile_col], row[prev_stile_col])
    if counter > max_counter:
        # reset counter check
        return 0
    return counter

_ = stiles_daily_df.apply(get_daily_counts, axis=1, max_counter=1000000, stile_col = 'entries', prev_stile_col = 'prev_entries')

In [ ]:
_ = stiles_daily_df.apply(get_daily_counts, axis=1, max_counter=1000000, stile_col = 'exits', prev_stile_col = 'prev_exits')

In [ ]:
#create daily entries / exits columns
stiles_daily_df["daily_entries"] = stiles_daily_df.apply(get_daily_counts, axis=1, max_counter=1000000, stile_col = 'entries', prev_stile_col = 'prev_entries')
stiles_daily_df["daily_exits"] = stiles_daily_df.apply(get_daily_counts, axis=1, max_counter=1000000, stile_col = 'exits', prev_stile_col = 'prev_exits')

In [ ]:
stiles_daily_df.head()

In [ ]:
#create station level df
station_daily_df = stiles_daily_df.groupby(['station','date'])[['daily_entries','daily_exits']].sum().reset_index()
station_daily_df.info()

In [ ]:
#convert date to dt
station_daily_df['date'] = pd.to_datetime(station_daily_df.date)

In [ ]:
station_daily_df.info()

In [ ]:
station_daily_df.head()

### A.2 Import & Clean Data: NWS Storm Warning Data
**Download storm warning csv from National Weather Service for August - October 2021**<br>
*note: attempted to retrieve via API but it was too complicated for me to do successfully*

In [ ]:
nws_df = pd.read_csv('/Users/oliviaoffutt/Desktop/Data_Science/Metis_EDA/MTA_Project/Raw_Data/NWS_alerts.csv')

In [ ]:
#check columns for white space
nws_df.columns

In [ ]:
#check nulls
nws_df.info()

In [ ]:
#convert date to dt
nws_df['date'] = pd.to_datetime(nws_df.date)
nws_df.info()

In [ ]:
nws_df.head()

### A.3 Merging DataFrames

In [ ]:
# ID days by whether or not they include a nws storm advisory (T/F)
nws_df_temp = nws_df[["date","event"]]
mta_station_df = pd.merge(station_daily_df, nws_df_temp, on='date', how='left')
mta_station_df = mta_station_df.drop_duplicates()
mta_station_df = mta_station_df.reset_index(drop=True)

In [ ]:
mta_station_df.head(3)

In [ ]:
mta_station_df[mta_station_df['event'] == 'FLASH FLOOD'].head(3)

In [ ]:
# ID days by weekday / weekend
mta_station_df['day'] = pd.to_datetime(mta_station_df['date']).dt.dayofweek
mta_station_df['week'] = pd.to_datetime(mta_station_df['date']).dt.week
mta_station_df.head(3)

In [ ]:
weekday_map = {0:True,
              1:True,
              2:True,
              3:True,
              4:True,
              5:False,
              6:False}
mta_station_df['weekday'] = mta_station_df['day'].map(weekday_map)

In [ ]:
mta_station_df.head(3)

### A.4 Review Clean DataFrames

In [ ]:
mta_station_df = mta_station_df[["station","event","date","day","weekday","week","daily_entries","daily_exits"]]
mta_station_df.head(1)

In [ ]:
nws_df = nws_df[["date","event","location","county"]]
nws_df.head(1)

## B. Data Analysis

For 2021 hurricane season (August 2021 - October 2021, ID stations with highest impacts to service during and following storm events**

*Output*                                     
- Plot daily entry data during storm weeks vs non-storm weeks
- ID top 10 stations that experience largest decreases in service by (a) magnitude (b) percent of average daily entries

### B.1 Explore Weather Data

In [ ]:
#import date formatter
import matplotlib.dates as mdates

years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')

dt_fmt = mdates.DateFormatter('%d')

#create necessary style variables
colors = sns.color_palette('Paired')[0:5]

In [ ]:
#create table: number of flash flood events by county
nws_df_bycounty = nws_df[['county','event']].groupby('county').count().sort_values('event', ascending = False).reset_index()
nws_df_bycounty

In [ ]:
#create chart: % share of flash flood events by county
storm_plt_1 = plt.pie(nws_df_bycounty['event'], labels = list(nws_df_bycounty['county']), colors = colors, autopct='%1.0f%%')
plt.title("% Share of Total Flood Events, by County, Aug-Oct 2021", fontsize = 12)

storm_plt_1

**FINDINGS**: More than two-thirds of flood events (72%) were located in Queens and Kings.

In [ ]:
#create table: number of flash flood events by date, county
nws_df_bydatecounty = nws_df[['date','county','event']].groupby(['date','county']).count().sort_values(['date','event']).reset_index()
nws_df_bydatecounty = nws_df_bydatecounty.astype({'date': 'str'})
nws_df_bydatecounty

In [ ]:
#create chart: number of flood events by date, county
storm_plt_2 = sns.barplot(x = 'date', y = 'event', hue = 'county', data = nws_df_bydatecounty, palette = 'Paired')
plt.xlabel("Date", fontsize = 10)
plt.ylabel("Number of Flood Warnings", fontsize = 10)
plt.title("Number of Flood Events, by Date and County, Aug-Oct 2021", fontsize = 12)

**FINDINGS**: 
- During the study period, there were two primary storm events that caused flooding, (a) 08-21 to 08-22 (b) 09-01 (Tropical Storm Ida)
- While the first storm had more sustained flooding, Ida had more severe flooding

### B.2 Explore Transit Data

*Method*
1. 8/21-22 storm event == Sat/Sun, wk 33; 9/1 storm event == Wednesday, wk 35
    - Can compare weekend storm event to average of all other weekends not in storm weekend
    - Can compare weekday storm event to average of all other weekdays not in storm week

In [ ]:
#ID True/False if a storm happened during that week
stormwk_map = {33: True,
              35: True}
mta_station_df['storm_wk'] = mta_station_df['week'].map(stormwk_map)
mta_station_df['storm_wk'] = mta_station_df.storm_wk.fillna(False)
mta_station_df.head()

In [ ]:
## create storm-week and non-storm-week totals for weekends and 5-day work weeks
weekly_sum = mta_station_df.groupby(['week','weekday','storm_wk']).daily_entries.sum().reset_index()
weekly_sum

In [ ]:
# renumber weeks
weekly_sum['week'] = weekly_sum['week'].transform(lambda x: x-29)

In [ ]:
weekly_sum

In [ ]:
#drop first and last week (incomplete data)
weekly_sum = weekly_sum.drop(0)
weekly_sum = weekly_sum.drop(27)
weekly_sum = weekly_sum.reset_index(drop = True)

In [ ]:
#create graph
plt.figure(figsize =(15,5))
sns.lineplot(x = 'week', y = 'daily_entries', hue = 'weekday', data = weekly_sum, palette = 'Paired')
plt.xlabel("Week Number (Week 4 and Week 6 are Storm Weeks)", fontsize = 10)
plt.ylabel("MTA Station Entries (tens of millions)", fontsize = 10)
plt.title("Total MTA Station Entries by Week, Aug-Oct 2021", fontsize = 12)
plt.legend(title='', loc='upper left', labels=['Weekend (Sa-Su)', 'Work Week (M-F)'])

**FINDINGS - Work Week**: 
- Only storm Ida (wk 6) has a significant drop in entries for the work week. This can be explained from 2 angles: 
    - (1) While both storms caused flooding, storm Ida had significantly more total floods and more extreme floods- it was an extreme weather event at the level of a tropical storm.
    - (2) Ida hit during the work week, while the August flooding events hit during the weekend
- Aug as a whole has lower entry numbers compared to Sep and Oct, perhaps due to summer holidays
- Week 6 experienced a particularly low drop, followed by sustained increases over the following two weeks where it them seems to return to a more sustained weekly rate. This might indicate that entry recovery for storm Ida (wk 6) may have taken 2 weeks, though this impact might be reduced significantly (even to zero) when controlled for normal annual traffic increases from summer to fall. 

**FINDINGS - Weekend**: 
- Weekend entries do not seem to be impacted by storms. This could be due to: (1) storm intensity/recovery, (2) lower MTA use overall, (3) and/or that stations most impacted by storm flooding are largely commuter stations.
- Even though week 4's flooding occurred over a weekend, there does not appear to be any impact.
- Week 8 has a suspiciously high number of entries - with more time I would look into whether this is an error or a real increase.

In [ ]:
## create storm-week and non-storm-week averages for weekends and 5-day work weeks
weekly_average = weekly_sum.groupby(['weekday','storm_wk']).daily_entries.mean().reset_index()
weekly_average['daily_entries'] = weekly_average['daily_entries'].transform(lambda x: round(x))
weekly_average

In [ ]:
#workweek nonstorm entries standard deviation
workweek_entries_std = weekly_sum[
    (weekly_sum['weekday'] == True) &
    (weekly_sum['storm_wk'] == False)
]['daily_entries'].std()
workweek_entries_std = round(workweek_entries_std,2)
workweek_entries_std

In [ ]:
#weekend nonstorm entries standard deviation
weekend_entries_std = weekly_sum[
    (weekly_sum['weekday'] == False) &
    (weekly_sum['storm_wk'] == False)
]['daily_entries'].std()
weekend_entries_std = round(weekend_entries_std,2)
weekend_entries_std

In [ ]:
#create graph
sns.barplot(x = 'weekday', y = 'daily_entries', hue = 'storm_wk', data = weekly_average, palette = 'Paired')
plt.xlabel("", fontsize = 10)
plt.ylabel("MTA Station Entries (tens of millions)", fontsize = 10)
plt.title("Weekend and Work Week Average Weekly Station Entries, Aug-Oct 2021", fontsize = 12)
plt.xticks([0,1],['Weekend (Sa-Su)','Work Week (M-F)'])

### Findings

- As observed, weekend average total mta entries during non-storm weeks vs storm weeks is nearly the same. The difference is a little over 600,000 entries, well below the standard deviation of 1.04 million entries
- On the other hand, the weekday average total mta entries during non-storm weeks vs storm weeks appears to be significantly different. The difference is 12.5 million entries, with the standard deviation being 1.89 million. 

In [ ]:
#pre-post storm ida (wk 6) comparison by station
#drop incomplete weeks and rename weeks starting at 1
mta_station_df = mta_station_df[
    (mta_station_df['week'] > 29) &
    (mta_station_df['week'] < 43)
]
mta_station_df['week'] = mta_station_df.week.transform(lambda x: x-29)

In [ ]:
mta_station_df.head()

In [ ]:
#weekly sums by station
station_wkly_sum_df = mta_station_df.groupby(['station','week'])['station','daily_entries','storm_wk'].sum('daily_entries').reset_index()

In [ ]:
station_wkly_sum_df.head(6)

In [ ]:
#Look only at weeks prior and after ida
ida_df = station_wkly_sum_df[
    (station_wkly_sum_df['week'] >4) &
    (station_wkly_sum_df['week'] <8)
]

In [ ]:
ida_df.head(6)

In [ ]:
#calculate entries decreases for wk 6 (storm) and 7 (post storm) compared to wk 5 (pre storm)
wk_5_entries = ida_df.groupby('station')[['daily_entries']].first().reset_index()

In [ ]:
ida_df = pd.merge(ida_df, wk_5_entries, on='station', how='left')

In [ ]:
ida_df['pre_strm_diff'] = ida_df['daily_entries_x'] - ida_df['daily_entries_y']

In [ ]:
ida_df['pre_strm_diff_pct'] = ida_df['pre_strm_diff'] / ida_df['daily_entries_y']

In [ ]:
#look at week 6 (storm week) decreases by station
ida_wk6_pct_top = ida_df[(ida_df['week'] == 6)].sort_values(by = ['pre_strm_diff_pct']).head(10)

In [ ]:
ida_wk6_pct_top_mrg = pd.merge(ida_wk6_pct_top, ida_df, on='station', how='left')

In [ ]:
ida_wk6_pct_top_mrg = ida_wk6_pct_top_mrg[['station','week_y','daily_entries_x_y','pre_strm_diff_y','pre_strm_diff_pct_y']]

In [ ]:
ida_wk6_pct_top_mrg.head()

In [ ]:
#create graph
sns.barplot(x = 'station', y = 'daily_entries_x_y', hue = 'week_y', data = ida_wk6_pct_top_mrg, palette = 'Paired')
plt.xlabel("Stations", fontsize = 10)
plt.ylabel("MTA Station Entries", fontsize = 10)
plt.title("Stations with Largest Percent Decrease in Ridership:" + "\n" + " Week 6 (Storm Week) Compared to Week 5 (Pre-Storm)", fontsize = 12)
plt.xticks(rotation=90)

In [ ]:
#create graph
sns.barplot(x = 'station', y = 'pre_strm_diff_pct', data = ida_wk6_pct_top, palette = 'Paired', ci=None)
plt.xlabel("Stations", fontsize = 10)
plt.ylabel("% Decrease in MTA Station Entries", fontsize = 10)
plt.title("Stations with Largest Percent Decrease in Ridership:" + "\n" + " Week 6 (Storm Week) Compared to Week 5 (Pre-Storm)", fontsize = 12)
plt.xticks(rotation=90)

In [ ]:
#look at week 7 (post storm week) decreases by station
ida_wk7_pct_top = ida_df[(ida_df['week'] == 7)].sort_values(by = ['pre_strm_diff_pct']).head(11).reset_index(drop = True)
ida_wk7_pct_top

In [ ]:
#drop outlier Orchard Beach
ida_wk7_pct_top = ida_wk7_pct_top.loc[1:10]
ida_wk7_pct_top

In [ ]:
ida_wk7_pct_top_mrg = pd.merge(ida_wk7_pct_top, ida_df, on='station', how='left')

In [ ]:
ida_wk7_pct_top_mrg = ida_wk7_pct_top_mrg[['station','week_y','daily_entries_x_y','pre_strm_diff_y','pre_strm_diff_pct_y']]

In [ ]:
ida_wk7_pct_top_mrg.head()

In [ ]:
#create graph
sns.barplot(x = 'station', y = 'daily_entries_x_y', hue = 'week_y', data = ida_wk7_pct_top_mrg, palette = 'Paired')
plt.xlabel("Stations", fontsize = 10)
plt.ylabel("MTA Station Entries", fontsize = 10)
plt.title("Stations with Largest Percent Decrease in Ridership:" + "\n" + " Week 7 (Post-Storm) Compared to Week 5 (Pre-Storm)", fontsize = 12)
plt.xticks(rotation=90)

In [ ]:
#create graph
sns.barplot(x = 'station', y = 'pre_strm_diff_pct', data = ida_wk7_pct_top, palette = 'Paired', ci=None)
plt.xlabel("Stations", fontsize = 10)
plt.ylabel("% Decrease in MTA Station Entries", fontsize = 10)
plt.title("Stations with Largest Percent Decrease in Ridership:" + "\n" + " Week 7 (Storm Week) Compared to Week 5 (Pre-Storm)", fontsize = 12)
plt.xticks(rotation=90)